   # Lab 2 - Solution

The template is given as starting point to help you get up to speed with loading file and understanding data.  
Use the template and answer each question in the appropriate cell.

Submit:
- eLearning
- Github: create new folder Lab-2. Commit your work under the folder

Naming: First-name_Last_name_ID_Lab_2.ipynb

## Dataset description

The dataset for Lab-2 is sample of the SBA dataset posted on Kaggle. 

The dataset is from the U.S. Small Business Administration (SBA)
The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market 
(SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment.
There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. 
However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.

More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied


In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## The function `describe_more` is data exploration utility

It is up to you if you want to use the function. If you find bugs, please let me know but try to fix them yourself.  
The function most definetely doesn't work with every input

In [2]:
"""
Created on Mon Mar 18 18:25:50 2019

@author: Uri Smashnov

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:200].to_dict())
                
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

## Load data

We assume data is located in the folder `data` where we run our notebook

In [3]:
data = pd.read_csv('./data/SBA_loans_small.zip')
data.head()

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BUFFALO,NY,14218,BANK OF AMERICA NATL ASSOC,NC,812111,2,7,1.0,0,0,1,1,Y,N,"$74,104.00",$0.00,CHGOFF,"$16,000.00","$8,000.00"
1,WEST MIFFLIN,PA,15122,CITIZENS BANK NATL ASSOC,RI,236115,2,1,1.0,0,1,1,1,Y,N,"$76,955.00",$0.00,CHGOFF,"$35,000.00","$17,500.00"
2,SAN DIEGO,CA,92126,MUFG UNION BANK NATL ASSOC,CA,442110,5,6,1.0,0,6,1,1,0,N,"$24,985.00",$0.00,CHGOFF,"$25,000.00","$12,500.00"
3,EDMONDS,WA,98026,BANK OF AMERICA NATL ASSOC,NC,333912,32,1,1.0,0,1,0,1,Y,N,"$7,265.00",$0.00,CHGOFF,"$5,000.00","$2,500.00"
4,PHILADELPHIA,PA,19120,"PNC BANK, NATIONAL ASSOCIATION",DE,812320,58,1,1.0,1,1,1,1,Y,N,"$9,972.00",$0.00,CHGOFF,"$10,000.00","$5,000.00"


In [4]:
data.shape

(448583, 20)

In [5]:
desc_df = describe_more(data)

In [6]:
pd.set_option('display.width', 200)
desc_df.head(n=200)

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,24662,object,1,30,"{'LOS ANGELES': 5737, 'HOUSTON': 5165, 'NEW YO..."
1,State,51,object,2,3,"{'CA': 65098, 'TX': 35003, 'NY': 28708, 'FL': ..."
2,Zip,28144,int64,1,5,"{10001: 465, 90015: 438, 90010: 369, 93401: 36..."
3,Bank,5099,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 43703, 'WELLS F..."
4,BankState,55,object,2,3,"{'CA': 59066, 'NC': 40088, 'IL': 32900, 'OH': ..."
5,NAICS,1289,int64,1,6,"{0: 100755, 722110: 13835, 722211: 9911, 81111..."
6,Term,378,int64,1,3,"{84: 114409, 60: 45031, 240: 42983, 120: 38667..."
7,NoEmp,473,int64,1,4,"{1: 76686, 2: 68846, 3: 45288, 4: 37017, 5: 29..."
8,NewExist,3,float64,3,3,"{1.0: 321926, 2.0: 126073, 0.0: 513}"
9,CreateJob,207,int64,1,4,"{0: 313701, 1: 31692, 2: 28798, 3: 14304, 4: 1..."


In [7]:
data.isna()

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
448579,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
448580,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
448581,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## Question 1

How many of the each target label are there? Target label is `MIS_Status` column. 

**Solution**:

In [8]:
data.groupby(by=["MIS_Status"])["MIS_Status"].count()

MIS_Status
CHGOFF     78779
P I F     369804
Name: MIS_Status, dtype: int64

In [9]:
pd.DataFrame(data.groupby(by=["MIS_Status"]))

,0,1
0,CHGOFF,City State Zip ...
1,P I F,City State Zip ...


In [10]:
data["MIS_Status"].isna().sum()

0

## Question 2

Encode target variable `MIS_Status` into `0` and `1`, when value `P I F` is mapped to `0` and `CHGOFF` is mapped to `1`.   

Replace values in the original column

**Solution:**

In [11]:
# Converting target variable from string to binary
data['MIS_Status'] = [1 if app == 'CHGOFF' else 0 for app in data.MIS_Status.values]

In [12]:
data.groupby(by=["MIS_Status"])["MIS_Status"].count()

MIS_Status
0    369804
1     78779
Name: MIS_Status, dtype: int64

## Question 3

Think about how to encode `ZIP` column? 

- Should we leave it as is? 
- Should we encode it using one-hot-encoding?
- Should we encode it using target encoder?

#### Question 3a

Copy original Pandas dataframe into new dataframe.  
Encode `ZIP` column using `from sklearn.preprocessing import OneHotEncoder`  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Check new dataframe. Is it good idea to use one-hot-encoder on `ZIP` column? 

**Solution:**

In [13]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
data_copy = data.copy()

"""
If trying to fit on full Dataframe, we get below error
enc.fit(data_copy[["Zip"]])

MemoryError: Unable to allocate 94.1 GiB for an array with shape (448583, 28144) and data type float64
"""

#Fit to sample of 5% of the data for demo only
data_copy = data_copy.sample(frac=.05)
enc.fit(data_copy[["Zip"]])
result = enc.transform(data_copy[["Zip"]])
ohe_columns = ["Zip"+"_"+str(x) for x in enc.categories_[0]]
result_df = pd.DataFrame(result, columns=ohe_columns)
data_copy = pd.concat([data_copy, result_df], axis=1)

In [14]:
print("New number of columns:", data_copy.shape[1])

New number of columns: 9819


**Conclusion**: One-hot-encoding has several limitations:
- It produces large number of columns
- It produces sparse features, and tree based algorithms don't work well with sparse features
- It cannot deal efficiently with unseen values
- .....

### Question 3b

Encode ZIP column using target **(mean)** encoding technique.
At this point `MIS_Status` column contains only 0 and 1.
Replace `ZIP` values with the mean of the response column (`MIS_Status`) for each value in a `ZIP` column. 

For example, if for ZIP=11222 there are total of 150 records. Out of which 80 records have label `0` and 70 records have label `1`, populate new column value for the record with replace with value:  
(0x80 + 1x70)/(80+70) = 0.467

The approach is not entirely  correct because of overfitting, but will give you general idea of one potential target encoding technique.  

Please see video on feature engineering topic posted in Module-2 for the explanation why frequency encoding is not ideal without modifications.

In [15]:
df1 = data.groupby(by=["Zip","MIS_Status"]).size().reset_index(name='counts')

In [16]:
df1.groupby(by="Zip").sum("counts")

,MIS_Status,counts
Zip,,
0,1,139
1,1,11
2,1,4
3,0,3
4,0,3
...,...,...
99919,0,1
99921,0,4
99925,0,2


In [17]:
df1[df1["MIS_Status"]==0][["Zip","counts"]][0:5]

,Zip,counts
0,0,26
2,1,10
4,2,2
6,3,3
7,4,3


In [18]:
df1[df1["MIS_Status"]==1][["Zip","counts"]][0:5]

,Zip,counts
1,0,113
3,1,1
5,2,2
10,7,1
12,8,3


**Solution**

Instead of replacing the values, new column is created. 

In [19]:
"""
Calculate frequency ratio of feature Zip in relation to target variable MIS_Status
"""
x = "Zip"
'''Dictionary to store each zip value count'''
total_val = pd.value_counts(data[x], dropna=True).to_dict()
'''Dictionary to store each zip value count for each label value'''
PIF_val = pd.value_counts(data[data["MIS_Status"]==0][x], dropna=True).to_dict()
CHGOFF_val = pd.value_counts(data[data["MIS_Status"]==1][x], dropna=True).to_dict()

'''Calculate Ratio'''
mean_dict = {}
for key,val in total_val.items():
    mean_dict[key] = (CHGOFF_val.get(key,0)*0 + PIF_val.get(key,0)*1)/(CHGOFF_val.get(key,0)+PIF_val.get(key,0))
    
'''Use zip/ratio dictionary to populate new column'''
data["Zip_freq"] = data["Zip"].map(mean_dict)        

In [20]:
'''Debug print'''
list(mean_dict.items())[0:10]

[(10001, 0.7634408602150538),
 (90015, 0.6506849315068494),
 (90010, 0.4905149051490515),
 (93401, 0.8688524590163934),
 (33166, 0.6871165644171779),
 (90021, 0.8359133126934984),
 (59601, 0.95625),
 (65804, 0.8802588996763754),
 (84115, 0.9096989966555183),
 (3801, 0.9342560553633218)]

In [21]:
'''Debug print'''
data[["Zip","Zip_freq"]].head()

,Zip,Zip_freq
0,14218,0.894737
1,15122,0.769231
2,92126,0.893401
3,98026,0.820896
4,19120,0.849057


In [22]:
zip_val = 15122
print("Validate ratio for zip code:",zip_val)
print("Number of records with MIS_Status zero for zip:",zip_val,"...", CHGOFF_val.get(zip_val))
print("Number of records with MIS_Status 1 for zip:",zip_val,"...",PIF_val.get(zip_val))
print("Ratio:",(CHGOFF_val.get(zip_val)*0+PIF_val.get(zip_val)*1)/(CHGOFF_val.get(zip_val)+PIF_val.get(zip_val)))

Validate ratio for zip code: 15122
Number of records with MIS_Status zero for zip: 15122 ... 6
Number of records with MIS_Status 1 for zip: 15122 ... 20
Ratio: 0.7692307692307693


## Question 4

Encode `LowDoc` column using `from sklearn import preprocessing.LabelEncoder`. Replace original column. 

**Note** - replace `LowDoc` with categorical variable

**Solution**

In [23]:
from sklearn.preprocessing import LabelEncoder
col = "LowDoc"
enc = LabelEncoder()
#enc.fit(list(data[col])+list(data[col]))
enc.fit(list(data[col]))
data[col] = enc.transform(data[[col]])

## Question 5

Fill all columns with missing values with `Unknown` or equivalent value.

```
# filling with Unknown class
df_clean = df.fillna("Unknown")
df_clean
```

In [24]:
values_to_fill = {}

for col in data.columns:
    if data[col].isna().any() == True:
        print("Processing column and column type:", col,data[col].dtype)
        
        if pd.api.types.is_numeric_dtype(data[col].dtype):
            values_to_fill[col] = 0
        else:
            values_to_fill[col] = "Unknown"
        

data.fillna(value=values_to_fill,inplace=True)

Processing column and column type: City object
Processing column and column type: State object
Processing column and column type: Bank object
Processing column and column type: BankState object
Processing column and column type: NewExist float64
Processing column and column type: RevLineCr object


In [25]:
values_to_fill

{'City': 'Unknown',
 'State': 'Unknown',
 'Bank': 'Unknown',
 'BankState': 'Unknown',
 'NewExist': 0,
 'RevLineCr': 'Unknown'}

## Question 6

Load same data into H2O-3 dataframe.  
Describe imported dataframe.

In [26]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "9"; OpenJDK Runtime Environment (build 9+181); OpenJDK 64-Bit Server VM (build 9+181, mixed mode)
  Starting server from /Users/usmashnov/var/virtual-env/ml-spring-2023/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/_z/jf3ghwdx1kg905xm5p1nktlh0000gp/T/tmpaw2l9gpp
  JVM stdout: /var/folders/_z/jf3ghwdx1kg905xm5p1nktlh0000gp/T/tmpaw2l9gpp/h2o_usmashnov_started_from_python.out
  JVM stderr: /var/folders/_z/jf3ghwdx1kg905xm5p1nktlh0000gp/T/tmpaw2l9gpp/h2o_usmashnov_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_usmashnov_6rapnt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [27]:
df_h = h2o.import_file('./data/SBA_loans_small.zip')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [28]:
df_h.describe()

Rows:448583
Cols:20

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
type,enum,enum,int,enum,enum,int,int,int,int,int,int,int,int,enum,enum,enum,int,enum,enum,enum
mins,,,0.0,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,,,
mean,,,53841.996368565015,,,398694.6823620155,110.78555807955271,11.40656912990461,1.2799479166666672,8.606342638931928,10.968233303535802,2760.593203487426,0.7566314372145178,,,,0.0,,,
maxs,,,99999.0,,,928120.0,527.0,9999.0,2.0,8800.0,8800.0,99999.0,2.0,,,,0.0,,,
sigma,,,31169.47801503186,,,263264.9256337996,78.85420615377285,72.25402961839185,0.4515142294208939,239.71012479114086,240.13097630419094,12783.22848127573,0.6458408354967028,,,,0.0,,,
zeros,,,139,,,100755,394,3318,513,313701,219984,103706,161424,,,,448581,,,
missing,13,6,0,755,757,0,0,0,71,0,0,0,0,130648,2037,0,2,0,0,0
0,BUFFALO,NY,14218.0,BANK OF AMERICA NATL ASSOC,NC,812111.0,2.0,7.0,1.0,0.0,0.0,1.0,1.0,Y,N,"$74,104.00",0.0,CHGOFF,"$16,000.00","$8,000.00"
1,WEST MIFFLIN,PA,15122.0,CITIZENS BANK NATL ASSOC,RI,236115.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,Y,N,"$76,955.00",0.0,CHGOFF,"$35,000.00","$17,500.00"
2,SAN DIEGO,CA,92126.0,MUFG UNION BANK NATL ASSOC,CA,442110.0,5.0,6.0,1.0,0.0,6.0,1.0,1.0,,N,"$24,985.00",0.0,CHGOFF,"$25,000.00","$12,500.00"


## Question 7

Use appropriate  method to fill missing values for the following columns:
- City
- State
- Bank
- BankState
- NewExist
- RevLineCr
- LowDoc

Use `fillna` method to fill missing values for numerical variables.

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-munging/fillnas.html

In [29]:
'''All Dataframe columns '''
df_h.columns

['City',
 'State',
 'Zip',
 'Bank',
 'BankState',
 'NAICS',
 'Term',
 'NoEmp',
 'NewExist',
 'CreateJob',
 'RetainedJob',
 'FranchiseCode',
 'UrbanRural',
 'RevLineCr',
 'LowDoc',
 'DisbursementGross',
 'BalanceGross',
 'MIS_Status',
 'GrAppv',
 'SBA_Appv']

In [30]:
df_h.columns_by_type(coltype="categorical")

[0.0, 1.0, 3.0, 4.0, 13.0, 14.0, 15.0, 17.0, 18.0, 19.0]

In [31]:
num_col = df_h.columns_by_type(coltype="numeric")
str_col = df_h.columns_by_type(coltype="string")
enum_col = df_h.columns_by_type(coltype="categorical")
all_columns = df_h.columns

In [32]:
# str_col contains indexes to the categorical and string columns
enum_col

[0.0, 1.0, 3.0, 4.0, 13.0, 14.0, 15.0, 17.0, 18.0, 19.0]

In [33]:
all_columns.index("City") in enum_col

True

**Solution**

Showing how to work around the problem with column name changes.

In [34]:
columns = ["City","State","Bank","BankState",
           "NewExist", "RevLineCr","LowDoc"]
num_col = df_h.columns_by_type(coltype="numeric")
str_col = df_h.columns_by_type(coltype="string")
enum_col = df_h.columns_by_type(coltype="categorical")
all_columns = df_h.columns

for col in columns:
    if all_columns.index(col)*1.0 in num_col:
        print("Fillna for numerical column:...", col)
        filled = df_h[col].fillna(method="forward",axis=1,maxlen=1)
        df_h[col] = filled
        '''Alternative way '''
        #df_h[df_h[col].isna(), col] = 0
    elif all_colum[/ns.index(col)*1.0 in enum_col:
        print("Fillna for categorical column:...", col)
        df_h[col] = df_h[col].ascharacter()
        df_h[df_h[col].isna(), col] = "Missing"
        df_h[col] = df_h[col].asfactor()
    elif all_columns.index(col)*1.0 in str_col:
        print("Fillna for string column:...", col)
        df_h[df_h[col].isna(), col] = "Missing"

Fillna for categorical column:... City
Fillna for categorical column:... State
Fillna for categorical column:... Bank
Fillna for categorical column:... BankState
Fillna for numerical column:... NewExist
Fillna for categorical column:... RevLineCr
Fillna for categorical column:... LowDoc


In [35]:
df_h.describe()

Rows:448583
Cols:20

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
type,enum,enum,int,enum,enum,int,int,int,int,int,int,int,int,enum,enum,enum,int,enum,enum,enum
mins,,,0.0,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,,,
mean,,,53841.996368565015,,,398694.6823620155,110.78555807955271,11.40656912990461,1.2799479166666672,8.606342638931928,10.968233303535802,2760.593203487426,0.7566314372145178,,,,0.0,,,
maxs,,,99999.0,,,928120.0,527.0,9999.0,2.0,8800.0,8800.0,99999.0,2.0,,,,0.0,,,
sigma,,,31169.47801503186,,,263264.9256337996,78.85420615377285,72.25402961839185,0.4515142294208939,239.71012479114086,240.13097630419094,12783.22848127573,0.6458408354967028,,,,0.0,,,
zeros,,,139,,,100755,394,3318,513,313701,219984,103706,161424,,,,448581,,,
missing,0,0,0,0,0,0,0,0,71,0,0,0,0,0,0,0,2,0,0,0
0,BUFFALO,NY,14218.0,BANK OF AMERICA NATL ASSOC,NC,812111.0,2.0,7.0,1.0,0.0,0.0,1.0,1.0,Y,N,"$74,104.00",0.0,CHGOFF,"$16,000.00","$8,000.00"
1,WEST MIFFLIN,PA,15122.0,CITIZENS BANK NATL ASSOC,RI,236115.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,Y,N,"$76,955.00",0.0,CHGOFF,"$35,000.00","$17,500.00"
2,SAN DIEGO,CA,92126.0,MUFG UNION BANK NATL ASSOC,CA,442110.0,5.0,6.0,1.0,0.0,6.0,1.0,1.0,Missing,N,"$24,985.00",0.0,CHGOFF,"$25,000.00","$12,500.00"


## Question 8

Perform target encoding on the following columns:
- ZIP
- City
- State

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/target-encoding.html


**Solution**

In [36]:
from h2o.estimators import H2OTargetEncoderEstimator

# Choose which columns to encode
encoded_columns = ["Zip","City","State"]
response = "MIS_Status"

df_h[encoded_columns] = df_h[encoded_columns].asfactor()

# Train a TE model
data_te = H2OTargetEncoderEstimator(#fold_column=fold_column,
                                    data_leakage_handling=None,
                                    blending=True,
                                    inflection_point=3,
                                    smoothing=10,
                                    noise=0.15,     # In general, the less data you have the more regularization you need
                                    seed=1234)

data_te.train(x=encoded_columns,
              y=response,
              training_frame=df_h)

# New target encoded dataset
df_h_te = data_te.transform(frame=df_h, as_training=True)

targetencoder Model Build progress: |████████████████████████████████████████████| (done) 100%


In [37]:
df_h_te.describe()

Rows:448583
Cols:23

,City_te,State_te,Zip_te,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
type,real,real,real,enum,enum,enum,enum,enum,int,int,int,int,int,int,int,int,enum,enum,enum,int,enum,enum,enum
mins,0.18252183632144942,0.5771587252515499,0.04020129751833368,,,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,,,
mean,0.8241730077331108,0.8243179504453161,0.8237187674579631,,,,,,398694.6823620155,110.78555807955271,11.40656912990461,1.2799479166666672,8.606342638931928,10.968233303535802,2760.593203487426,0.7566314372145178,,,,0.0,,,
maxs,1.1484827592169922,1.089542243978024,1.145076333410229,,,,,,928120.0,527.0,9999.0,2.0,8800.0,8800.0,99999.0,2.0,,,,0.0,,,
sigma,0.11953959630630948,0.09660749549896634,0.1282079402729322,,,,,,263264.9256337996,78.85420615377285,72.25402961839185,0.4515142294208939,239.71012479114086,240.13097630419094,12783.22848127573,0.6458408354967028,,,,0.0,,,
zeros,0,0,0,,,,,,100755,394,3318,513,313701,219984,103706,161424,,,,448581,,,
missing,0,0,0,0,0,0,0,0,0,0,0,71,0,0,0,0,0,0,0,2,0,0,0
0,0.9229630235661851,0.8673192560938519,0.9624461352333614,BUFFALO,NY,14218,BANK OF AMERICA NATL ASSOC,NC,812111.0,2.0,7.0,1.0,0.0,0.0,1.0,1.0,Y,N,"$74,104.00",0.0,"$16,000.00","$8,000.00",CHGOFF
1,0.6110918766855461,0.7866627672590444,0.7055639175674366,WEST MIFFLIN,PA,15122,CITIZENS BANK NATL ASSOC,RI,236115.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,Y,N,"$76,955.00",0.0,"$35,000.00","$17,500.00",CHGOFF
2,0.8866187019563224,0.8540197719745422,0.9328028263305289,SAN DIEGO,CA,92126,MUFG UNION BANK NATL ASSOC,CA,442110.0,5.0,6.0,1.0,0.0,6.0,1.0,1.0,Missing,N,"$24,985.00",0.0,"$25,000.00","$12,500.00",CHGOFF


In [38]:
df_h_te.head()

City_te,State_te,Zip_te,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
0.922963,0.867319,0.962446,BUFFALO,NY,14218,BANK OF AMERICA NATL ASSOC,NC,812111,2,7,1,0,0,1,1,Y,N,"$74,104.00",0,"$16,000.00","$8,000.00",CHGOFF
0.611092,0.786663,0.705564,WEST MIFFLIN,PA,15122,CITIZENS BANK NATL ASSOC,RI,236115,2,1,1,0,1,1,1,Y,N,"$76,955.00",0,"$35,000.00","$17,500.00",CHGOFF
0.886619,0.85402,0.932803,SAN DIEGO,CA,92126,MUFG UNION BANK NATL ASSOC,CA,442110,5,6,1,0,6,1,1,Missing,N,"$24,985.00",0,"$25,000.00","$12,500.00",CHGOFF
1.00348,1.00314,0.967239,EDMONDS,WA,98026,BANK OF AMERICA NATL ASSOC,NC,333912,32,1,1,0,1,0,1,Y,N,"$7,265.00",0,"$5,000.00","$2,500.00",CHGOFF
0.694558,0.734355,0.727892,PHILADELPHIA,PA,19120,"PNC BANK, NATIONAL ASSOCIATION",DE,812320,58,1,1,1,1,1,1,Y,N,"$9,972.00",0,"$10,000.00","$5,000.00",CHGOFF
0.643896,0.815043,0.810158,SAINT LOUIS,MO,63139,U.S. BANK NATIONAL ASSOCIATION,OH,442299,29,4,1,0,4,0,1,Y,N,"$53,654.00",0,"$20,000.00","$10,000.00",CHGOFF
0.76532,0.703029,0.647662,LAFAYETTE,LA,70507,HOME BANK,LA,621320,82,5,2,0,0,1,1,N,Y,"$91,500.00",0,"$91,500.00","$77,775.00",CHGOFF
0.762355,0.776973,0.729021,VALENCIA,CA,91402,BANK OF AMERICA NATL ASSOC,NC,337110,49,3,1,10,3,1,1,T,N,"$20,000.00",0,"$20,000.00","$10,000.00",CHGOFF
0.580516,0.719489,0.709626,BROOKLYN,NY,11237,JPMORGAN CHASE BANK NATL ASSOC,IL,561510,29,6,1,1,6,0,1,N,N,"$40,400.00",0,"$40,400.00","$20,200.00",CHGOFF
0.834749,0.931922,0.67148,Richmond,VA,23238,SUNTRUST BANK,GA,812199,32,4,2,4,0,0,1,Y,N,"$47,499.00",0,"$25,000.00","$12,500.00",CHGOFF
